In [175]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

---

## Data merging and cleaning

In [176]:
# Read the CSV housing_data file from the Resources folder into a Pandas DataFrame
housing_data = Path("../Resources/housing_data2.csv")
income_data = Path("../Resources/income_data.csv")
merged_crime_data = Path("../Resources/merged_crime_data.csv")
zipcode_data = Path("../Resources/us_city_zipcode_data.csv")
housing_df = pd.read_csv(housing_data, dtype={'zip_code': str})
income_df = pd.read_csv(income_data,  dtype={'zipcode': str})
crime_df = pd.read_csv(merged_crime_data)
zipcode_df = pd.read_csv(zipcode_data)


In [177]:
# View the columns of four dataframes to check if 'zipcode' exists for the merging
print(housing_df.columns)
print(income_df.columns)
print(crime_df.columns)
print(zipcode_df.columns)

Index(['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street',
       'city', 'state', 'zip_code', 'house_size', 'prev_sold_date'],
      dtype='object')
Index(['state', 'zipcode', 'total_pop', 'total_income', 'country',
       'avg_income'],
      dtype='object')
Index(['states', 'cities', 'population', 'violent_crime', 'robbery',
       'prop_crime', 'burglary', 'vehicle_theft', 'total_crime',
       'tot_violent_crime', 'tot_prop_crim', 'arson'],
      dtype='object')
Index(['country code', 'postal code', 'place name', 'admin name1',
       'admin code1', 'admin name2', 'admin code2', 'latitude', 'longitude'],
      dtype='object')


In [178]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        object 
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(7), object(5)
memory usage: 203.8+ MB


In [179]:
housing_df = housing_df.rename(columns= {
    'zip_code': 'zipcode'
})

In [180]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zipcode         object 
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(7), object(5)
memory usage: 203.8+ MB


In [181]:
# Drop unecessary columns in zipcode_df for the merge to main data
zipcode_df_drop = zipcode_df.drop(columns=['country code', 'admin name1', 'admin name2', 'admin code2', 'latitude', 'longitude'])

zipcode_df_drop.head()

,postal code,place name,admin code1
0,99547,Atka,AK
1,99660,Saint Paul Island,AK
2,99509,Anchorage,AK
3,99523,Anchorage,AK
4,99524,Anchorage,AK


In [182]:
# Rename colums in zipcode
zipcode_renamed_df = zipcode_df_drop.rename(columns= {
    "postal code": "zipcode",
    "place name": "city",
    "admin code1": "state_code",
}
)
zipcode_renamed_df.head()

,zipcode,city,state_code
0,99547,Atka,AK
1,99660,Saint Paul Island,AK
2,99509,Anchorage,AK
3,99523,Anchorage,AK
4,99524,Anchorage,AK


In [183]:
zipcode_renamed_df['zipcode'] = zipcode_renamed_df['zipcode'].astype(str)

In [184]:
# Merge housing, income data
housing_income_df = housing_df.merge(income_df, how='left', on = 'zipcode')

# Merge the result with zipcode_renamed_df on 'zipcode'
housing_income_with_city_df = housing_income_df.merge(zipcode_renamed_df, on='zipcode', how='left')

# Show the result
housing_income_with_city_df.tail()

,brokered_by,status,price,bed,bath,acre_lot,street,city_x,state_x,zipcode,house_size,prev_sold_date,state_y,total_pop,total_income,country,avg_income,city_y,state_code
2228227,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354,3600.0,2022-03-25,WA,11180.0,831326.0,USA,74358.318426,Richland,WA
2228228,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354,1616.0,2022-03-25,WA,11180.0,831326.0,USA,74358.318426,Richland,WA
2228229,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354,3200.0,2022-03-24,WA,11180.0,831326.0,USA,74358.318426,Richland,WA
2228230,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354,933.0,2022-03-24,WA,11180.0,831326.0,USA,74358.318426,Richland,WA
2228231,108243.0,sold,580000.0,5.0,3.0,0.31,307704.0,Richland,Washington,99354,3615.0,2022-03-23,WA,11180.0,831326.0,USA,74358.318426,Richland,WA


In [185]:
housing_income_with_city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2228232 entries, 0 to 2228231
Data columns (total 19 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city_x          object 
 8   state_x         object 
 9   zipcode         object 
 10  house_size      float64
 11  prev_sold_date  object 
 12  state_y         object 
 13  total_pop       float64
 14  total_income    float64
 15  country         object 
 16  avg_income      float64
 17  city_y          object 
 18  state_code      object 
dtypes: float64(10), object(9)
memory usage: 323.0+ MB


In [186]:
housing_income_with_city_df['sqft_lot'] = housing_income_with_city_df['acre_lot'] * 43560

In [187]:
# Drop unessary columns of the main data before merging
housing_income_with_city_drop_df = housing_income_with_city_df.drop(columns=['brokered_by', 'status', 'street', 'state_y', 'city_y', 'total_pop', 'total_income', 'country', 'acre_lot'])

# Rename columns
housing_income_with_city_drop_df = housing_income_with_city_drop_df.rename(columns= {
    "bath": "bathrooms",
    "bed": "bedrooms",
    "house_size": "sqft_living",
    "city_x": "city",
    "state_x": "state",

}
)
housing_income_with_city_drop_df.head()

,price,bedrooms,bathrooms,city,state,zipcode,sqft_living,prev_sold_date,avg_income,state_code,sqft_lot
0,105000.0,3.0,2.0,Adjuntas,Puerto Rico,00601,920.0,NaN,NaN,NaN,5227.2
1,80000.0,4.0,2.0,Adjuntas,Puerto Rico,00601,1527.0,NaN,NaN,NaN,3484.8
2,67000.0,2.0,1.0,Juana Diaz,Puerto Rico,00795,748.0,NaN,NaN,NaN,6534.0
3,145000.0,4.0,2.0,Ponce,Puerto Rico,00731,1800.0,NaN,NaN,NaN,4356.0
4,65000.0,6.0,2.0,Mayaguez,Puerto Rico,00680,NaN,NaN,NaN,NaN,2178.0


In [188]:
housing_income_with_city_drop_df

,price,bedrooms,bathrooms,city,state,zipcode,sqft_living,prev_sold_date,avg_income,state_code,sqft_lot
0,105000.0,3.0,2.0,Adjuntas,Puerto Rico,00601,920.0,NaN,NaN,NaN,5227.2
1,80000.0,4.0,2.0,Adjuntas,Puerto Rico,00601,1527.0,NaN,NaN,NaN,3484.8
2,67000.0,2.0,1.0,Juana Diaz,Puerto Rico,00795,748.0,NaN,NaN,NaN,6534.0
3,145000.0,4.0,2.0,Ponce,Puerto Rico,00731,1800.0,NaN,NaN,NaN,4356.0
4,65000.0,6.0,2.0,Mayaguez,Puerto Rico,00680,NaN,NaN,NaN,NaN,2178.0
...,...,...,...,...,...,...,...,...,...,...,...
2228227,359900.0,4.0,2.0,Richland,Washington,99354,3600.0,2022-03-25,74358.318426,WA,14374.8
2228228,350000.0,3.0,2.0,Richland,Washington,99354,1616.0,2022-03-25,74358.318426,WA,4356.0
2228229,440000.0,6.0,3.0,Richland,Washington,99354,3200.0,2022-03-24,74358.318426,WA,21780.0
2228230,179900.0,2.0,1.0,Richland,Washington,99354,933.0,2022-03-24,74358.318426,WA,3920.4


In [189]:
# Crop unesscessary columns in crime data
crime_dropped_df = crime_df.drop(columns=['total_crime',
       'tot_violent_crime', 'tot_prop_crim', 'arson', 'states'])

# Rename columns
crime_renamed_df = crime_dropped_df.rename(columns= {
    "cities": "city"
}
)
crime_renamed_df.head()

,city,population,violent_crime,robbery,prop_crime,burglary,vehicle_theft
0,"Abington Township, Montgomery County","55,731",197.4,70.0,1979.1,296.1,32.3
1,Albany,"51,084",86.1,45.0,3092.9,438.5,184.0
2,Alexandria,"48,449",1682.2,293.1,7492.4,2010.4,379.8
3,Aliso Viejo,"48,999",87.8,12.2,847.0,208.2,26.5
4,Altamonte Springs,"42,296",335.7,82.8,3057.0,427.9,165.5


In [190]:
crime_renamed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           975 non-null    object 
 1   population     975 non-null    object 
 2   violent_crime  856 non-null    object 
 3   robbery        975 non-null    float64
 4   prop_crime     873 non-null    object 
 5   burglary       973 non-null    object 
 6   vehicle_theft  975 non-null    object 
dtypes: float64(1), object(6)
memory usage: 53.4+ KB


In [191]:
# Function to clean and convert columns to numeric
def clean_and_convert_to_numeric(col):
    col = col.replace({',': ''}, regex=True)  # Remove commas
    return pd.to_numeric(col, errors='coerce')  # Convert to numeric, invalid values become NaN

# Remove commas from the population column and convert it to numeric
crime_renamed_df['population'] = crime_renamed_df['population'].replace({',': ''}, regex=True)
crime_renamed_df['population'] = pd.to_numeric(crime_renamed_df['population'], errors='coerce')

# List of columns to clean and convert
columns_to_convert = ['population', 'violent_crime', 'prop_crime', 'burglary', 'vehicle_theft']

# Apply the cleaning function to the selected columns
crime_renamed_df[columns_to_convert] = crime_renamed_df[columns_to_convert].apply(clean_and_convert_to_numeric)

# Check the result
print(crime_renamed_df.dtypes)  # Check the data types of columns
print(crime_renamed_df.head())  # Display the first few rows

city              object
population         int64
violent_crime    float64
robbery          float64
prop_crime       float64
burglary         float64
vehicle_theft    float64
dtype: object
                                   city  population  violent_crime  robbery  \
0  Abington Township, Montgomery County       55731          197.4     70.0   
1                                Albany       51084           86.1     45.0   
2                            Alexandria       48449         1682.2    293.1   
3                           Aliso Viejo       48999           87.8     12.2   
4                     Altamonte Springs       42296          335.7     82.8   

   prop_crime  burglary  vehicle_theft  
0      1979.1     296.1           32.3  
1      3092.9     438.5          184.0  
2      7492.4    2010.4          379.8  
3       847.0     208.2           26.5  
4      3057.0     427.9          165.5  


In [192]:
crime_renamed_df['population'].unique()

array([  55731,   51084,   48449,   48999,   42296,   46404,   59271,
         56132,   46482,   43046,   49827,   57295,   43675,   44033,
         54785,   47111,   45570,   41477,   55510,   45563,   48763,
         42769,   51678,   56933,   40150,   41016,   45037,   47250,
         43070,   57953,   41367,   47620,   52848,   40608,   43888,
         55765,   59043,   50878,   40253,   52811,   44884,   54796,
         59658,   58145,   52213,   41687,   40277,   51430,   47223,
         40339,   40008,   50132,   56801,   50038,   52867,   46633,
         52051,   46167,   49856,   58586,   43991,   43474,   47560,
         55825,   45761,   42068,   42034,   54651,   45267,   58345,
         44690,   53431,   44817,   47590,   42815,   59856,   40053,
         48435,   55062,   48588,   40785,   59769,   41799,   49470,
         48976,   43121,   43324,   43722,   57228,   56126,   54318,
         58644,   50795,   56470,   47427,   48185,   43600,   47004,
         42343,   58

In [193]:
# Convert the relevant columns to float
columns_to_convert = ['population', 'violent_crime', 'prop_crime', 'burglary', 'vehicle_theft']

# Apply pd.to_numeric to each column, coercing errors to NaN
crime_renamed_df[columns_to_convert] = crime_renamed_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Check the DataFrame dtypes to confirm the conversion
print(crime_renamed_df.dtypes)

city              object
population         int64
violent_crime    float64
robbery          float64
prop_crime       float64
burglary         float64
vehicle_theft    float64
dtype: object


In [194]:
crime_renamed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           975 non-null    object 
 1   population     975 non-null    int64  
 2   violent_crime  856 non-null    float64
 3   robbery        975 non-null    float64
 4   prop_crime     873 non-null    float64
 5   burglary       973 non-null    float64
 6   vehicle_theft  975 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 53.4+ KB


In [195]:
# Count occurrences of each city in the 'city' column
city_counts = crime_renamed_df['city'].value_counts()

# Get the cities that appear more than once
cities_more_than_once = city_counts[city_counts > 1].index

# Filter rows in crime_renamed_df where 'city' appears more than once
filtered_df = crime_renamed_df[crime_renamed_df['city'].isin(cities_more_than_once)]

# Sort the filtered DataFrame in descending order based on a specific column, e.g., 'population'
filtered_df_sorted = filtered_df.sort_values(by='city', ascending=False)

# Display the sorted DataFrame
print(filtered_df_sorted)

            city  population  violent_crime  robbery  prop_crime  burglary  \
655   Wilmington       72088         1703.5    657.5      5304.6    1681.3   
870   Wilmington      109370          565.1    238.6      5403.7    1548.9   
652  Westminster       91908          309.0     95.7      3128.1     507.0   
868  Westminster      109461          259.5     46.6      2585.4     360.9   
333       Warren       41355          674.6    246.6      4860.4    1963.5   
..           ...         ...            ...      ...         ...       ...   
2     Alexandria       48449         1682.2    293.1      7492.4    2010.4   
665   Alexandria      145892          166.6     94.6      2049.5     192.6   
359       Albany       78512         1035.5    285.3      6369.7    1793.4   
360       Albany       98187          816.8    253.6      4420.1     903.4   
1         Albany       51084           86.1     45.0      3092.9     438.5   

     vehicle_theft  
655          506.3  
870          341.0  


In [196]:
# Assuming your dataframe is named `housing_income_city_crime_df2` and has a 'city' column
crime_renamed_df['city'] = crime_renamed_df['city'].str.split(',').str[0]

In [197]:
# Groupby crime record

crime_renamed_df2 = crime_renamed_df.groupby('city')[['population', 'violent_crime', 'robbery', 'prop_crime',
       'burglary', 'vehicle_theft']].sum()

crime_renamed_df2 = crime_renamed_df2.reset_index()

# Display the result
print(crime_renamed_df2.head())

                city  population  violent_crime  robbery  prop_crime  \
0            Abilene      119886          393.7    105.9      3664.3   
1  Abington Township       55731          197.4     70.0      1979.1   
2              Akron      198390          886.6    290.8      5057.7   
3            Alameda       75467          212.0    106.0      2507.1   
4             Albany      227783         1938.4    583.9     13882.7   

   burglary  vehicle_theft  
0     865.0          142.6  
1     296.1           32.3  
2    1728.4          363.4  
3     392.2          390.9  
4    3135.3          626.1  


In [198]:
# Calculate total crime by summing the relevant columns
crime_renamed_df2['total_crime'] = crime_renamed_df2['violent_crime'] + \
                                    crime_renamed_df2['robbery'] + \
                                    crime_renamed_df2['prop_crime'] + \
                                    crime_renamed_df2['burglary'] + \
                                    crime_renamed_df2['vehicle_theft']

# Calculate crime rate per capita
crime_renamed_df2['crime_rate_per_capita'] = crime_renamed_df2['total_crime'] / crime_renamed_df2['population']

# Display the result (optional, for checking the first few rows)
print(crime_renamed_df2[['city', 'total_crime', 'crime_rate_per_capita']].head())

                city  total_crime  crime_rate_per_capita
0            Abilene       5171.5               0.043137
1  Abington Township       2574.9               0.046202
2              Akron       8326.9               0.041972
3            Alameda       3608.2               0.047812
4             Albany      20166.4               0.088533


In [205]:
# Drop unesscessary columns in crime data
crime_renamed_cleaned = crime_renamed_df2.drop(columns=['violent_crime',
       'robbery', 'prop_crime', 'burglary', 'vehicle_theft', 'total_crime', 'population'])

In [206]:
crime_renamed_cleaned.city.value_counts()

city
Abilene               1
Pasco                 1
Palm Bay              1
Palm Beach Gardens    1
Palm Desert           1
                     ..
Gilroy                1
Glendale              1
Glendora              1
Glenview              1
Yuma                  1
Name: count, Length: 888, dtype: int64

In [207]:
crime_renamed_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   city                   888 non-null    object 
 1   crime_rate_per_capita  888 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.0+ KB


In [208]:
# Show columns of the two dats
print(housing_income_with_city_drop_df.columns)
print(crime_renamed_cleaned.columns)

Index(['price', 'bedrooms', 'bathrooms', 'city', 'state', 'zipcode',
       'sqft_living', 'prev_sold_date', 'avg_income', 'state_code',
       'sqft_lot'],
      dtype='object')
Index(['city', 'crime_rate_per_capita'], dtype='object')


In [209]:
# Merge the main housing data with crime data
housing_income_city_crime_df1 = housing_income_with_city_drop_df.merge(crime_renamed_cleaned, how="left", on="city")
housing_income_city_crime_df1.head()

,price,bedrooms,bathrooms,city,state,zipcode,sqft_living,prev_sold_date,avg_income,state_code,sqft_lot,crime_rate_per_capita
0,105000.0,3.0,2.0,Adjuntas,Puerto Rico,00601,920.0,NaN,NaN,NaN,5227.2,NaN
1,80000.0,4.0,2.0,Adjuntas,Puerto Rico,00601,1527.0,NaN,NaN,NaN,3484.8,NaN
2,67000.0,2.0,1.0,Juana Diaz,Puerto Rico,00795,748.0,NaN,NaN,NaN,6534.0,NaN
3,145000.0,4.0,2.0,Ponce,Puerto Rico,00731,1800.0,NaN,NaN,NaN,4356.0,NaN
4,65000.0,6.0,2.0,Mayaguez,Puerto Rico,00680,NaN,NaN,NaN,NaN,2178.0,NaN


In [210]:
# Review the data
housing_income_city_crime_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2228232 entries, 0 to 2228231
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   price                  float64
 1   bedrooms               float64
 2   bathrooms              float64
 3   city                   object 
 4   state                  object 
 5   zipcode                object 
 6   sqft_living            float64
 7   prev_sold_date         object 
 8   avg_income             float64
 9   state_code             object 
 10  sqft_lot               float64
 11  crime_rate_per_capita  float64
dtypes: float64(7), object(5)
memory usage: 204.0+ MB


In [211]:
# Check for NaN values
print(housing_income_city_crime_df1.isna().sum())

price                       1541
bedrooms                  482867
bathrooms                 513321
city                        2457
state                          8
zipcode                      299
sqft_living               570034
prev_sold_date            736147
avg_income                151820
state_code                126759
sqft_lot                  325589
crime_rate_per_capita    1314459
dtype: int64


In [212]:
# Filter rows where 'city' is NaN
nan_city_rows = housing_income_city_crime_df1[housing_income_city_crime_df1['city'].isna()]

# Display the result
print(nan_city_rows)

            price  bedrooms  bathrooms city           state zipcode  \
2523      25000.0       NaN        NaN  NaN  Virgin Islands     NaN   
2526      90000.0       NaN        NaN  NaN  Virgin Islands     NaN   
2527     120000.0       NaN        NaN  NaN  Virgin Islands     NaN   
2561     649000.0       NaN        NaN  NaN  Virgin Islands     NaN   
2606      55000.0       NaN        NaN  NaN  Virgin Islands     NaN   
...           ...       ...        ...  ...             ...     ...   
2154880  850000.0       NaN        NaN  NaN      California   95043   
2163483  175000.0       NaN        NaN  NaN      California   95469   
2176111  720000.0       3.0        2.0  NaN          Nevada   89449   
2178256   19191.0       NaN        NaN  NaN      California   96021   
2225446  467500.0       4.0        2.0  NaN      Washington   99027   

         sqft_living prev_sold_date    avg_income state_code    sqft_lot  \
2523             NaN            NaN           NaN        NaN     10890.

In [213]:
# Drop rows where any NaN values exist
housing_income_city_crime_df2 = housing_income_city_crime_df1.dropna()

# Check the resulting DataFrame
print(housing_income_city_crime_df2.head())

          price  bedrooms  bathrooms       city     state zipcode  \
32546  105000.0       3.0        2.0    Hampton  New York   12837   
32597  325000.0       4.0        3.0  Cambridge  New York   12816   
32599  719900.0       3.0        3.0      Salem  New York   12873   
32600  215000.0       5.0        2.0      Salem  New York   12865   
32602  119900.0       1.0        1.0      Salem  New York   12865   

       sqft_living prev_sold_date    avg_income state_code   sqft_lot  \
32546       1352.0     2018-12-18  47894.285714         NY   183823.2   
32597       1760.0     2021-09-08  58557.004831         NY    85813.2   
32599       3034.0     2011-01-25  58325.641026         NY   405108.0   
32600       1932.0     2016-02-23  54669.090909         NY  2343963.6   
32602        642.0     2014-06-11  54669.090909         NY    91476.0   

       crime_rate_per_capita  
32546               0.032271  
32597               0.037274  
32599               0.045241  
32600               0.

In [214]:
# Recheck data after handling NA
housing_income_city_crime_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500618 entries, 32546 to 2228231
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   price                  500618 non-null  float64
 1   bedrooms               500618 non-null  float64
 2   bathrooms              500618 non-null  float64
 3   city                   500618 non-null  object 
 4   state                  500618 non-null  object 
 5   zipcode                500618 non-null  object 
 6   sqft_living            500618 non-null  float64
 7   prev_sold_date         500618 non-null  object 
 8   avg_income             500618 non-null  float64
 9   state_code             500618 non-null  object 
 10  sqft_lot               500618 non-null  float64
 11  crime_rate_per_capita  500618 non-null  float64
dtypes: float64(7), object(5)
memory usage: 49.7+ MB


In [215]:
housing_income_city_crime_df2

,price,bedrooms,bathrooms,city,state,zipcode,sqft_living,prev_sold_date,avg_income,state_code,sqft_lot,crime_rate_per_capita
32546,105000.0,3.0,2.0,Hampton,New York,12837,1352.0,2018-12-18,47894.285714,NY,183823.2,0.032271
32597,325000.0,4.0,3.0,Cambridge,New York,12816,1760.0,2021-09-08,58557.004831,NY,85813.2,0.037274
32599,719900.0,3.0,3.0,Salem,New York,12873,3034.0,2011-01-25,58325.641026,NY,405108.0,0.045241
32600,215000.0,5.0,2.0,Salem,New York,12865,1932.0,2016-02-23,54669.090909,NY,2343963.6,0.045241
32602,119900.0,1.0,1.0,Salem,New York,12865,642.0,2014-06-11,54669.090909,NY,91476.0,0.045241
...,...,...,...,...,...,...,...,...,...,...,...,...
2228227,359900.0,4.0,2.0,Richland,Washington,99354,3600.0,2022-03-25,74358.318426,WA,14374.8,0.064415
2228228,350000.0,3.0,2.0,Richland,Washington,99354,1616.0,2022-03-25,74358.318426,WA,4356.0,0.064415
2228229,440000.0,6.0,3.0,Richland,Washington,99354,3200.0,2022-03-24,74358.318426,WA,21780.0,0.064415
2228230,179900.0,2.0,1.0,Richland,Washington,99354,933.0,2022-03-24,74358.318426,WA,3920.4,0.064415


In [216]:
housing_income_city_crime_df2['city'].nunique()

786

In [217]:
housing_income_city_crime_df2.city = housing_income_city_crime_df2.city.apply(lambda x: x.strip())
city_stats = housing_income_city_crime_df2.groupby("city")["city"].agg('count').sort_values(ascending=False)
city_stats.tail(1000)

/var/folders/24/7rhj3_jn39lg7m0d74t2dp5w0000gn/T/ipykernel_64518/4135065224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_income_city_crime_df2.city = housing_income_city_crime_df2.city.apply(lambda x: x.strip())


city
Houston        15976
Tucson          7555
Phoenix         7382
Los Angeles     6899
Dallas          6703
               ...  
Waltham            1
Revere             1
New Britain        1
Taunton            1
Hoboken            1
Name: city, Length: 786, dtype: int64

In [218]:
city_stats.describe()

count      786.000000
mean       636.918575
std       1066.400923
min          1.000000
25%        150.250000
50%        338.500000
75%        678.500000
max      15976.000000
Name: city, dtype: float64

In [231]:
len(city_stats[city_stats<=638])

576

In [232]:
# Move less than 100 datapoint location to Other

city_stats_less_than_100 = city_stats[city_stats<=100]
city_stats_less_than_100

len (housing_income_city_crime_df2.city.unique())


667

In [233]:
housing_income_city_crime_df2.city = housing_income_city_crime_df2.city.apply(lambda x: 'Other' if x in city_stats_less_than_100 else x)
len(housing_income_city_crime_df2.city.unique())

/var/folders/24/7rhj3_jn39lg7m0d74t2dp5w0000gn/T/ipykernel_64518/2521677634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_income_city_crime_df2.city = housing_income_city_crime_df2.city.apply(lambda x: 'Other' if x in city_stats_less_than_100 else x)


667

In [234]:
housing_income_city_crime_df2.columns

Index(['price', 'bedrooms', 'bathrooms', 'city', 'state', 'zipcode',
       'sqft_living', 'prev_sold_date', 'avg_income', 'state_code', 'sqft_lot',
       'crime_rate_per_capita', 'price_per_sqft_living'],
      dtype='object')

In [235]:
from datetime import datetime
# Get the current year

# Calculate price per sqft foot
housing_income_city_crime_df2['price_per_sqft_living'] = housing_income_city_crime_df2['price'] / housing_income_city_crime_df2['sqft_living']

/var/folders/24/7rhj3_jn39lg7m0d74t2dp5w0000gn/T/ipykernel_64518/3483871165.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_income_city_crime_df2['price_per_sqft_living'] = housing_income_city_crime_df2['price'] / housing_income_city_crime_df2['sqft_living']


In [236]:
housing_income_city_crime_df2.columns

Index(['price', 'bedrooms', 'bathrooms', 'city', 'state', 'zipcode',
       'sqft_living', 'prev_sold_date', 'avg_income', 'state_code', 'sqft_lot',
       'crime_rate_per_capita', 'price_per_sqft_living'],
      dtype='object')

In [244]:
# Combine city & state code to one column
housing_income_city_crime_df2['city_state'] = housing_income_city_crime_df2['city'] + ', ' + housing_income_city_crime_df2['state_code'].astype(str)

# Drop unecessary columns 
housing_income_city_crime_df3 = housing_income_city_crime_df2.drop(columns=['sqft_lot', 'zipcode', 'state_code'])

housing_income_city_crime_df3.head()

/var/folders/24/7rhj3_jn39lg7m0d74t2dp5w0000gn/T/ipykernel_64518/3341074062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_income_city_crime_df2['city_state'] = housing_income_city_crime_df2['city'] + ', ' + housing_income_city_crime_df2['state_code'].astype(str)


,price,bedrooms,bathrooms,city,state,sqft_living,prev_sold_date,avg_income,crime_rate_per_capita,price_per_sqft_living,city_state
32546,105000.0,3.0,2.0,Hampton,New York,1352.0,2018-12-18,47894.285714,0.032271,77.662722,"Hampton, NY"
32597,325000.0,4.0,3.0,Cambridge,New York,1760.0,2021-09-08,58557.004831,0.037274,184.659091,"Cambridge, NY"
32599,719900.0,3.0,3.0,Salem,New York,3034.0,2011-01-25,58325.641026,0.045241,237.277521,"Salem, NY"
32600,215000.0,5.0,2.0,Salem,New York,1932.0,2016-02-23,54669.090909,0.045241,111.283644,"Salem, NY"
32602,119900.0,1.0,1.0,Salem,New York,642.0,2014-06-11,54669.090909,0.045241,186.760125,"Salem, NY"


In [144]:
housing_merge_df3 = housing_merge_df2[(housing_merge_df2['bathrooms'] != 0)]
housing_merge_df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19943 entries, 0 to 19949
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           19943 non-null  float64
 1   bathrooms       19943 non-null  float64
 2   sqft_living     19943 non-null  int64  
 3   floors          19943 non-null  float64
 4   yr_built        19943 non-null  int64  
 5   zipcode         19943 non-null  int64  
 6   avg_income      19943 non-null  float64
 7   city            19943 non-null  object 
 8   violent_crime   19943 non-null  float64
 9   robbery         19943 non-null  float64
 10  prop_crime      19943 non-null  float64
 11  age_of_house    19943 non-null  int64  
 12  price_per_sqft  19943 non-null  float64
dtypes: float64(8), int64(4), object(1)
memory usage: 2.1+ MB


In [145]:
print(housing_merge_df3.isna().sum())

price             0
bathrooms         0
sqft_living       0
floors            0
yr_built          0
zipcode           0
avg_income        0
city              0
violent_crime     0
robbery           0
prop_crime        0
age_of_house      0
price_per_sqft    0
dtype: int64


In [71]:
# Export clean data to csv 
housing_merge_df3.to_csv('../Resources/housing_merge3.csv', index=False)